# Carrefour Analytics Project

Este é um projeto de Data Science que faz uso de um conjunto de dados de vendas disponibilizadas pelo **Grupo Carrefour** no contexto de aplicação para a vaga de **ANALYTICS ENGINEER SR**. O projeto encontra-se dividido em 3 etapas:

* Analytics
* Data Science
* Pipeline de dados

## Data Science

O objetivo desta etapa é gerar um conjunto de dados que auxilie o time comercial no entendimento da cesta de compras e possibilite a criação de combos promocionais. Para isso, devemos calcular a "relevância" de cada produto ou grupo de produtos no período de análise. O significado de relevância aqui adotado será o de *suporte*, que é uma medida estatística que, no nosso contexto, indica a proporção de vendas de um produto ou grupo de produtos dentro do universo total de vendas analisado. A participação em venda será tratada de forma *booleana*, ou seja, será considerada apenas a presença de um determinado item na venda, desconsiderando-se sua quantidade. Iniciamos o projeto por esta etapa pois, nela será gerado um conjunto de dados necessário para o desenvolvimento do relatório solicitado na etapa de Analytics.

Começamos o projeto importando os pacotes e dados que foram previamente carregados em um repositório do [github](https://github.com/mateusmelo821/Carrefour-Analytics-Project/tree/main/data). Os dados foram divididos em 2 partes devido a limitações da ferramenta que não suportava arquivos de tamanho superior a 100MB.

In [21]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')

df_p1 = pd.read_csv("https://raw.githubusercontent.com/mateusmelo821/Carrefour-Analytics-Project/main/data/data_p1.csv")
df_p1.tail()

,periodo,id_clube,socio,canal,ticket,departamento,item_id,item_descricao,item_unidade,receita_bruta,margem
439293,2022-01-11,6170,45752210215738782,PISO,4575221021573878220220111,BAZAR E TÊXTIL,1241243,KIT 2 CALCINHAS BIKINI TAM. M LOBA,4,185.33,32.33
439294,2022-01-11,6170,45765100802861021,PISO,4576510080286102120220111,BAZAR E TÊXTIL,1228874,KIT 2 CALCINHAS BIKINI TAM. P LOBA,2,80.41,2.13
439295,2022-01-11,6170,45752210215738782,PISO,4575221021573878220220111,BAZAR E TÊXTIL,1241242,KIT 2 CALCINHAS BIKINI TAM. P LOBA,2,93.35,16.85
439296,2022-01-11,6170,45765100801921925,PISO,4576510080192192520220111,BAZAR E TÊXTIL,1227375,KIT 2 CUECAS BOX ALGODAO REF 781/G,1,41.36,10.76
439297,2022-01-11,6170,45752100198379059,PISO,4575210019837905920220111,BAZAR E TÊXTIL,1227375,KIT 2 CUECAS BOX ALGODAO REF 781/G,1,41.36,10.76


In [22]:
df_p2 = pd.read_csv("https://raw.githubusercontent.com/mateusmelo821/Carrefour-Analytics-Project/main/data/data_p2.csv")
df_p2.tail()

,periodo,id_clube,socio,canal,ticket,departamento,item_id,item_descricao,item_unidade,receita_bruta,margem
439293,2022-01-12,7780,45752200200689222,PISO,4575220020068922220220112,HIGIENE E LIMPEZA,1235849,YPE CONCENTRAD BLUE AMACIANTE 1.5LT,1,21.36,5.05
439294,2022-01-12,7780,45752100207181231,PISO,4575210020718123120220112,HIGIENE E LIMPEZA,1235849,YPE CONCENTRAD BLUE AMACIANTE 1.5LT,1,21.36,5.05
439295,2022-01-12,7780,45765100803107283,PISO,4576510080310728320220112,HIGIENE E LIMPEZA,1235849,YPE CONCENTRAD BLUE AMACIANTE 1.5LT,1,21.36,5.05
439296,2022-01-12,7780,45752220207119702,PISO,4575222020711970220220112,HIGIENE E LIMPEZA,1235849,YPE CONCENTRAD BLUE AMACIANTE 1.5LT,1,21.36,5.05
439297,2022-01-12,7780,45752100210188207,PISO,4575210021018820720220112,HIGIENE E LIMPEZA,1235849,YPE CONCENTRAD BLUE AMACIANTE 1.5LT,1,21.36,5.05


Agora podemos mesclar os dois subconjuntos e extrair algumas informações gerais sobre o conjunto de dados original.

In [23]:
df = pd.concat([df_p1, df_p2], ignore_index=True)
df.shape

(878596, 11)

In [24]:
df.dtypes

periodo            object
id_clube            int64
socio               int64
canal              object
ticket             object
departamento       object
item_id             int64
item_descricao     object
item_unidade        int64
receita_bruta     float64
margem            float64
dtype: object

In [25]:
df.nunique()

periodo               3
id_clube             42
socio             71889
canal                 2
ticket            75378
departamento          8
item_id           10661
item_descricao    10251
item_unidade        143
receita_bruta     16015
margem            13799
dtype: int64

In [26]:
df['periodo'].unique()

array(['2022-01-10', '2022-01-11', '2022-01-12'], dtype=object)

Temos 878.596 observações de 11 variáveis. Nesta etapa, estamos interessados principalmente nas variáveis:
* ticket
* periodo
* item_id
* item_descricao

Estamos considerando o **ticket** como o identificador da cesta de compras. A nossa base contém informações de 75.378 cestas. O **periodo** refere-se a data da venda e, como temos dados para apenas 3 dias, ele não será considerado. O **item_id** é o identificador do item de descrição igual a **item_descricao**. Como temos uma quantidade maior de **item_id** do que **item_descricao**, concluímos que existem produtos com múltiplos identificadores. Assim sendo, utilizaremos a coluna **item_descricao** para identificar os produtos. Como temos uma quantidade elevada de dados, serão considerados apenas os 100 produtos mais vendidos.

Vamos filtrar os dados com base nessas considerações, mantendo apenas as colunas **ticket** e **item_descricao**.

In [27]:
df_grouped = df.groupby('item_descricao')['ticket'].count().reset_index()
df_grouped = df_grouped.sort_values('ticket', ascending=False, ignore_index=True).head(100)
df_grouped.tail()

,item_descricao,ticket
95,REQUEIJAO LIGHT MEMBERS MARK 500G,917
96,QUEIJO MINAS FRESCAL FAZENDA KG,913
97,BATATA ESPECIAL KG KG,909
98,BROCOLIS 1KG DAUCY,901
99,ABACATE KG,899


In [28]:
df = df[df['item_descricao'].isin(df_grouped['item_descricao'])]
df = df[['ticket', 'item_descricao']].reset_index(drop=True)
df.tail()

,ticket,item_descricao
169434,4575210021802609420220112,PAPEL TOALHA KITCHEN JUMB FD C360FL
169435,4575221019548999220220112,PAPEL TOALHA KITCHEN JUMB FD C360FL
169436,4575212019898566520220112,PAPEL TOALHA KITCHEN JUMB FD C360FL
169437,4576610090438117520220112,PAPEL TOALHA KITCHEN JUMB FD C360FL
169438,4575220020068922220220112,PAPEL TOALHA KITCHEN JUMB FD C360FL


In [29]:
df['ticket'].nunique()

55366

Agora iremos alterar o formato do dataframe para melhor adequação ao algoritmo que será utilizado no cálculo de suporte. Para isso, iremos transformar cada item da coluna **item_descricao** em uma nova coluna e, para cada par de ticket e produto, teremos uma célula indicando se o produto fez parte daquela venda.

In [30]:
df_format = pd.crosstab(df['ticket'], df['item_descricao'])
df_format = df_format>0
df_format.tail()

item_descricao,ABACATE KG,ACUC REF UNIAO 1KG,AGUA SANITARIA 5LT MEMBERS MARK,ALCOOL LIQ BACT 70 INPM TOPFORM 1L,AZEITE EV ITALIANO ORGANICO 1L MM,AZEITE RAHMA EXTRA VIRGEM 1LT,BACON MINI CARDEAL PECA KG,BANANA NANICA KG,BANANA PRATA KG,BATATA ESPECIAL KG KG,...,SAL REF CISNE 1KG,SHAMPOO ALTA MODA BB CREAM 1LT,SUCO DE LARANJA INTEGRAL 2L,SUCO NATURAL ONE LARANJA 2L,SUCO NATURAL ONE UVA/MACA 2L,SUPER CANDIDA 5LT AGUA SANITARIA,TODDY ORIGINAL 1.800KG,TOMATE PELADO CIRIO 3 X 400G,VEJA LIMPEZA PESADA CLORO 2X1LT,WAFFLE TRAD 525G FORNO DE MINAS
ticket,,,,,,,,,,,,,,,,,,,,,
4576610090696798920220112,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4576610090696830020220112,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4576610090696883920220112,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4576610090696961320220110,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4576610090696961320220112,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Agora iremos calcular o suporte dos produtos e grupos de produtos usando o algoritmo [apriori](https://en.wikipedia.org/wiki/Apriori_algorithm). Vamos utilizar a implementação do pacote [mlxtend](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/). Utilizaremos um valor de suporte mínimo de 0.3% para capturarmos combos com mais de 2 produtos.

In [31]:
df_suporte = apriori(df_format, min_support=0.003, use_colnames=True).sort_values('support', ascending=False)
df_suporte.head()

,support,itemsets
87,0.240220,(SACOLA PLASTICA MM BRANCA)
73,0.105913,(PAPEL TOALHA KITCHEN JUMB FD C360FL)
58,0.077394,(MM LAVA ROUPAS 5LTS ULTIMATE CLEAN)
75,0.069700,(PH NEVE FT 20M L32P28UN)
69,0.069032,(PAO FRANCES MEMBERS MARK)


In [32]:
print("Quantidade de combos relevantes com mais de 2 produtos: ", sum(df_suporte['itemsets'].apply(lambda x: len(x)>2)))
print("Quantidade de combos relevantes com 2 produtos: ", sum(df_suporte['itemsets'].apply(lambda x: len(x)==2)))

Quantidade de combos relevantes com mais de 2 produtos:  3
Quantidade de combos relevantes com 2 produtos:  354


Agora iremos calcular a confiança (*confidence*) e a alavancagem (*lift*). Como estamos interessados em formar combos promocionais, serão considerados apenas os grupos de produtos com alavancagem superior a 1.

In [39]:
df_combos = association_rules(df_suporte, metric='lift', min_threshold=1)[['antecedents', 'consequents', 'antecedent support',
                                                                           'support', 'confidence', 'lift']]
df_combos.tail()

,antecedents,consequents,antecedent support,support,confidence,lift
549,(LIMPOL 6X500ML DETERG LIQ NATURAL),(MON BIJOU PUREZA 5LT AMAC DILUIDO),0.044883,0.003016,0.067203,3.098063
550,(CR DE LEITE PIRACANJUBA TP 200G),(LEITE FERMENTADO YAKULT 6X80G),0.031445,0.003016,0.095922,2.294087
551,(LEITE FERMENTADO YAKULT 6X80G),(CR DE LEITE PIRACANJUBA TP 200G),0.041813,0.003016,0.072138,2.294087
552,(DORITOS NACHO 300GR),(PAPEL TOALHA KITCHEN JUMB FD C360FL),0.022107,0.003016,0.136438,1.288203
553,(PAPEL TOALHA KITCHEN JUMB FD C360FL),(DORITOS NACHO 300GR),0.105913,0.003016,0.028479,1.288203


Foram encontrados 554 grupos com alavancagem superior a 1. Vamos verificar e persistir o resultado de grupos com mais de 2 produtos.

In [43]:
df_combos[df_combos['antecedents'].apply(lambda x: len(x)>1)]

,antecedents,consequents,antecedent support,support,confidence,lift
408,"(PH NEVE FT 20M L32P28UN, SACOLA PLASTICA MM B...",(PAPEL TOALHA KITCHEN JUMB FD C360FL),0.012932,0.003468,0.268156,2.531847
409,"(SACOLA PLASTICA MM BRANCA, PAPEL TOALHA KITCH...",(PH NEVE FT 20M L32P28UN),0.022396,0.003468,0.154839,2.221508
462,"(BATATA ESPECIAL KG KG, CENOURA KG)",(CEBOLA KG KG),0.006340,0.003269,0.515670,23.479078
463,"(BATATA ESPECIAL KG KG, CEBOLA KG KG)",(CENOURA KG),0.006773,0.003269,0.482667,13.262195
464,"(CENOURA KG, CEBOLA KG KG)",(BATATA ESPECIAL KG KG),0.006123,0.003269,0.533923,32.520569
534,"(PH PERSONAL VIP FD 40 ROLOSX30M, SACOLA PLAST...",(PAPEL TOALHA KITCHEN JUMB FD C360FL),0.010403,0.003034,0.291667,2.753823
535,"(SACOLA PLASTICA MM BRANCA, PAPEL TOALHA KITCH...",(PH PERSONAL VIP FD 40 ROLOSX30M),0.022396,0.003034,0.135484,2.047271


In [41]:
df_combos[df_combos['antecedents'].apply(lambda x: len(x)>1)].to_csv("resultado_combos_3_produtos.csv", index=False)

Os 3 combos considerados relevantes com mais de 2 itens tiveram alavancagem superior a 1. O combo formado pelos produtos CENOURA KG, CEBOLA KG KG e BATATA ESPECIAL KG KG obteve um excelente resultado. Na compra dos 2 primeiros, a chance de compra do terceiro é superior a 53% e alavancagem é maior que 32 vezes.

Agora iremos deixar os combos com 2 produtos é um formato mais adequado para criarmos o relatório da etapa de Analytics.

In [46]:
df_combos_2 = df_combos[(df_combos['antecedents'].apply(lambda x: len(x)==1))&(df_combos['consequents'].apply(lambda x: len(x)==1))]
df_combos_2['antecedents'] = df_combos_2['antecedents'].apply(lambda x: list(x)[0])
df_combos_2['consequents'] = df_combos_2['consequents'].apply(lambda x: list(x)[0])
df_combos_2.head()

,antecedents,consequents,antecedent support,support,confidence,lift
0,PAO FRANCES MEMBERS MARK,SACOLA PLASTICA MM BRANCA,0.069032,0.017592,0.254840,1.060864
1,SACOLA PLASTICA MM BRANCA,PAO FRANCES MEMBERS MARK,0.240220,0.017592,0.073233,1.060864
2,PH NEVE FT 20M L32P28UN,PAPEL TOALHA KITCHEN JUMB FD C360FL,0.069700,0.016924,0.242809,2.292525
3,PAPEL TOALHA KITCHEN JUMB FD C360FL,PH NEVE FT 20M L32P28UN,0.105913,0.016924,0.159789,2.292525
4,PH PERSONAL VIP FD 40 ROLOSX30M,PAPEL TOALHA KITCHEN JUMB FD C360FL,0.066178,0.016183,0.244541,2.308882


In [47]:
df_combos_2.to_csv('produtos_relevantes.csv', index=False)

## Analytics